In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
import bs4
import requests
import time

In [6]:
class MvpaAdmin :

    
    def __init__(self) :
    
        global target 
        
        target = input("What is your target? Just put number. (1.Calender 2.Ceremony 3.Hero 4.History 5.Monument)")
        chrome_options = Options()
        chrome_options.add_argument('--start-maximized')
        
        self.driver = webdriver.Chrome('./chromedriver', options = chrome_options)
        self.alert = Alert(self.driver)
    
        self.driver.execute_script('window.open("about:blank", "_blank");')
        
        self.tabs = self.driver.window_handles
        
        self.driver.switch_to.window(self.tabs[0])
        self.driver.get('http://175.114.94.38/login.do')
                
        id_input = self.driver.find_element(By.NAME, 'j_username')
        pw_input = self.driver.find_element(By.NAME, 'j_password')
        
        id_input.send_keys('id')
        pw_input.send_keys('pw')
        
        self.driver.find_element(By.ID, 'loginBtnClick').click()
        self.driver.get('http://175.114.94.38/adm/notice/scheduleList.do')
        self.contentsCheck()
        
    def targetTranslate(self):
        
        if target == '1':
            return 'H202'
        elif target == '2':
            return 'H1023'
        elif target == '3':
            return 'H6001'
        elif target == '4':
            return 'H6002'
        elif target == '5':
            return 'H6003'
        else :
            raise Exception("Wrong Order!")
        
        
    def contentsCheck(self):
        
        menu_id = self.targetTranslate()
        
        self.driver.find_element(By.CLASS_NAME, 'tab02').click()
        self.driver.find_element(By.ID, menu_id).click()
        
        if target == '1':
            self.calenderCheck()
        elif target == '2' :
            self.ceremonyCheck()
        elif target == '3':
            self.heroCheck()
        elif target == '4' :
            self.historyCheck()
        elif target == '5' :
            self.monumentCheck()
        else :
            raise Exception("Wrong Order!")
        
    def calenderCheck(self) :
        return 0
        
    def ceremonyCheck(self) :
        return 0
    
    def heroCheck(self) :
        
        birth_error = False
        
        hero_no = self.driver.find_element(By.CLASS_NAME, 'txt_c01').text
        q = int(hero_no) // 10
        r = int(hero_no) % 10 
        
        check = 1
        
        for  _ in range(q + 1) :
            
            if _ < 14 :
                xpath = '//*[@id="container"]/div[3]/div[3]/div[3]/div[1]/a[' + str(_ % 10 + 3) + ']'
            
                if (_ + 1) % 10 == 0 :
                    self.driver.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[3]/div[3]/div[1]/a[12]').click()
                    continue
                else :
                    self.driver.find_element(By.XPATH, xpath).click()
                    continue
            
            
            ls = self.driver.find_element(By.ID, 'listForm').find_elements(By.CLASS_NAME, 'detailWarp')
            
            for index in range (len(ls)) :
                
                c_path = '//*[@id="listForm"]/div/table/tbody/tr['+ str(index + 1) +']/td[2]'
                print(check, self.driver.find_element(By.XPATH, c_path).text)
                check += 1
                
                self.driver.find_element(By.ID, 'listForm').find_elements(By.CLASS_NAME, 'detailWarp')[index].click()
                attr = self.driver.find_element(By.ID, 'container')\
                        .find_element(By.CLASS_NAME, 'contents')\
                        .find_element(By.CLASS_NAME, 'cont')\
                        .find_element(By.CLASS_NAME, 'tbl_style01')\
                        .find_element(By.TAG_NAME, 'tbody')\
                        .find_element(By.TAG_NAME, 'tr')\
                        .find_element(By.TAG_NAME, 'td').text
                        
                if '독립' in attr :

                    name = self.driver.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[3]/div[2]/table/tbody/tr[3]/td').text
                    birth = self.driver.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[3]/div[3]/table/tbody/tr[1]/td').text
                    fbirth = self.birth_definer(self.valid_birth(birth),birth)
                    
                    try : 
                        
                        try :
                            int(fbirth[:4])
                            birth_error = False
                        except :
                            print('birth error! on ',name)
                            birth_error = True
                        
                        self.driver.find_element(By.CLASS_NAME, 'btn_wrap')\
                            .find_element(By.ID, 'update').click()
                        '''
                        if birth_error != True and int(birth) <= 1945 and :
                            print(birth)
                        '''
                        self.driver.switch_to.window(self.tabs[1])
                        self.driver.get('https://e-gonghun.mpva.go.kr/user/ContribuMeritList.do?goTocode=20002')
                        self.driver.find_element(By.XPATH, '//*[@id="nameKo"]').send_keys(name)
                        self.driver.find_element(By.XPATH, '//*[@id="frm"]/div/div/div/div[2]/a[2]').click()
                        
                        record_no = int(self.driver.find_element(By.XPATH, '//*[@id="subcontents"]/div/div/div[1]/ul[1]/li/div/ul/li[2]/strong').text)
                        self.driver.find_element(By.XPATH, '//*[@id="searchNum"]/option[4]').click()
                        if (record_no < 1) :
                            print(name, " 공훈록 정보 없음")
                            birth_error = False
                            self.driver.switch_to.window(self.driver.window_handles[0])
                            
                        elif (record_no == 1 and birth_error != True) :
                            self.driver.find_element(By.XPATH, '//*[@id="namekoDiv0"]').click()
                            self.heroMaker(name, fbirth)
                            
                        elif (record_no == 1 and birth_error == True) :
                            self.driver.find_element(By.XPATH, '//*[@id="namekoDiv0"]').click()
                            print('error on birth, but record found ',name)
                            self.heroMaker(name,fbirth)
                            
                        elif (record_no > 1 and birth_error == True) :
                            print(name, ' 생년월일 error, pass')
                            self.driver.switch_to.window(self.driver.window_handles[0])
                            
                        elif (record_no > 1 and birth_error != True) :
                            
                            hero_ls = {}
                            b_no = 0
                            
    
                            for i in range(record_no) :

                                path = '//*[@id="subcontents"]/div/div/div[2]/table/tbody/tr[' + str(i + 1) + ']/td[4]'
                                namepath = '//*[@id="namekoDiv'+ str(i) +'"]'
                                pagename = self.driver.find_element(By.XPATH, namepath).text
                                if (pagename == name and fbirth[:4] in (self.driver.find_element(By.XPATH, path)).text) :
                                    hero_ls[b_no] = namepath
                                    b_no += 1
                                elif (pagename == name and fbirth[-8:-4] in (self.driver.find_element(By.XPATH, path)).text) :
                                    hero_ls[b_no] = namepath
                                    b_no += 1

                                
                            if b_no < 1 :
                                print(name, " 생년월일 오류")
                                self.driver.switch_to.window(self.driver.window_handles[0])
                            elif b_no > 1 :
                                print(name, '동명동년')
                                self.driver.switch_to.window(self.driver.window_handles[0])
                            else :
                                self.driver.find_element(By.XPATH, hero_ls[0]).click()
                                self.heroMaker(name, fbirth)
                                
                        self.driver.back() 
                    
                    except :
                        print(name)
                        
                    
                self.driver.back()
                
            xpath = '//*[@id="container"]/div[3]/div[3]/div[3]/div[1]/a[' + str(_ % 10 + 3) + ']'
            
            if (_ + 1) % 10 == 0 :
                self.driver.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[3]/div[3]/div[1]/a[12]').click()
            else :
                self.driver.find_element(By.XPATH, xpath).click()

    def gonghunMaker(self, text) :
        
        text = text.replace('또한','\n\n또한').replace('그후','\n\n그후').replace('그리하여','\n\n그리하여').replace('그러나','\n\n그러나').replace('사람이다.','사람이다.\n\n').replace('한편','\n\n한편').replace('따라서','\n\n따라서')
        return text
    
    def heroMaker(self, name, fbirth) :
        
        text = ""
        gonghun = self.driver.find_elements(By.ID, "bm_cont")
        for line in gonghun :
            text += line.text + "\n"
            
        text = self.gonghunMaker(text)
        
        mvpa_no = self.driver.find_element(By.XPATH,'//*[@id="personInfo"]/div[1]/span/small').text[:-4]
        
        self.driver.switch_to.window(self.tabs[0])
        self.driver.find_element(By.XPATH, '//*[@id="chronology"]').clear()
        self.driver.find_element(By.XPATH, '//*[@id="chronology"]').send_keys(text)
        
        try :
            dyear = int(fbirth[-8:-4])
            dmonth = int(fbirth[-4:-2])
            dday = int(fbirth[-2:])
        except :
            print('error! ' + fbirth)
        
        if(self.hero_definer(dyear, dmonth, dday) == True) :
            self.driver.find_element(By.XPATH, '//*[@id="independentHeroTypeCdNWA01"]').click()
        else :
            self.driver.find_element(By.XPATH, '//*[@id="independentHeroTypeCdNWA02"]').click()
        
        try :
            if('No image' in self.driver.find_element(By.XPATH,'//*[@id="fileName"]/a').text) :
                self.driver.find_element(By.XPATH, '//*[@id="upThumbFile"]').send_keys('C:/Users/USER/Desktop/Duane/Python/MVPA/인물_기본.png')
        except :
            self.driver.find_element(By.XPATH, '//*[@id="upThumbFile"]').send_keys('C:/Users/USER/Desktop/Duane/Python/MVPA/인물_기본.png')
            
        try :
            self.driver.find_element(By.XPATH, '//*[@id="upImgFilePreview_1676"]/img').text
        except :
            self.driver.find_element(By.XPATH, '//*[@id="upImgFile"]').send_keys('C:/Users/USER/Desktop/Duane/Python/MVPA/배경_기본.png')    
        
        self.driver.find_element(By.XPATH, "//*[@id='contSelect']").click()

        self.driver.switch_to.window(self.driver.window_handles[2])
        self.find_contents(name)
        
        no_contents = False    
            
        if len(self.driver.find_element(By.XPATH, '//*[@id="choiceContTr"]').find_elements(By.TAG_NAME, 'td')) == 1:
            
            if self.contents_maker(name) == False :
                no_contents = True
            else :
                self.driver.find_element(By.XPATH, '//*[@id="searchValue"]').clear()
                self.find_contents(name)
        
        if no_contents == True :
            print(name, "콘텐츠 없음.")
            self.driver.close()
        
        else :
            self.driver.find_element(By.XPATH, '//*[@id="listForm"]/div[2]/div/button').click()
        
        self.driver.switch_to.window(self.driver.window_handles[0])
        adr = 'https://e-gonghun.mpva.go.kr/user/ContribuReportDetail.do?goTocode=20001&pageTitle=Report&mngNo=' + mvpa_no
        self.driver.find_element(By.XPATH, '//*[@id="linkTitle"]').clear()
        self.driver.find_element(By.XPATH, '//*[@id="linkTitle"]').send_keys('공훈전자사료관(공적조서)')
        self.driver.find_element(By.XPATH, '//*[@id="linkUrl"]').clear()
        self.driver.find_element(By.XPATH, '//*[@id="linkUrl"]').send_keys(adr)
        self.driver.find_element(By.XPATH, '//*[@id="update"]').click()
        self.alert.accept()
        
        self.driver.back() 
    
    def find_contents(self, name) :
        
        self.driver.find_element(By.XPATH, '//*[@id="searchValue"]').send_keys(name)
        self.driver.find_element(By.XPATH, '//*[@id="search_btn"]').click()
        
        contents = self.driver.find_element(By.XPATH, '//*[@id="listDiv"]/table/tbody').find_elements(By.CLASS_NAME, 'button')
        
        for cont in range (len(contents)) : 
            self.driver.find_element(By.XPATH, '//*[@id="listDiv"]/table/tbody').find_elements(By.CLASS_NAME, 'button')[cont].click()
    
    def historyCheck(self) :
        
        hist_quant = self.driver.find_element(By.XPATH,'//*[@id="container"]/div[3]/div[3]/div[2]/span/span').text
        q = int(hist_quant) // 10
        r = int(hist_quant) % 10 
        
        for  _ in range(q + 1) :
            
            # if _ < 14 :
            #     xpath = '//*[@id="container"]/div[3]/div[3]/div[3]/div[1]/a[' + str(_ % 10 + 3) + ']'
            
            #     if (_ + 1) % 10 == 0 :
            #         self.driver.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[3]/div[3]/div[1]/a[12]').click()
            #         continue
            #     else :
            #         self.driver.find_element(By.XPATH, xpath).click()
            #         continue
            # This code is for hero.
            
            
            ls = self.driver.find_element(By.ID, 'listForm').find_elements(By.CLASS_NAME, 'detailWarp')
            
            for index in range (len(ls)) :
            
                self.driver.find_element(By.ID, 'listForm').find_elements(By.CLASS_NAME, 'detailWarp')[index].click()
                attr = self.driver.find_element(By.ID, 'container')\
                        .find_element(By.CLASS_NAME, 'contents')\
                        .find_element(By.CLASS_NAME, 'cont')\
                        .find_element(By.CLASS_NAME, 'tbl_style01')\
                        .find_element(By.TAG_NAME, 'tbody')\
                        .find_element(By.TAG_NAME, 'tr')\
                        .find_element(By.TAG_NAME, 'td').text
                        
                if '독립' in attr :

                    name = self.driver.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[3]/div[2]/table/tbody/tr[3]/td').text
                    birth = self.driver.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[3]/div[3]/table/tbody/tr[1]/td').text
                    fbirth = self.birth_definer(self.valid_birth(birth),birth)
                    
                    try : 
                        
                        try :
                            int(fbirth[:4])
                            birth_error = False
                        except :
                            print('birth error! on ',name)
                            birth_error = True
                        
                        self.driver.find_element(By.CLASS_NAME, 'btn_wrap')\
                            .find_element(By.ID, 'update').click()
                        '''
                        if birth_error != True and int(birth) <= 1945 and :
                            print(birth)
                        '''
                        self.driver.switch_to.window(self.tabs[1])
                        self.driver.get('https://e-gonghun.mpva.go.kr/user/ContribuMeritList.do?goTocode=20002')
                        self.driver.find_element(By.XPATH, '//*[@id="nameKo"]').send_keys(name)
                        self.driver.find_element(By.XPATH, '//*[@id="frm"]/div/div/div/div[2]/a[2]').click()
                        
                        record_no = int(self.driver.find_element(By.XPATH, '//*[@id="subcontents"]/div/div/div[1]/ul[1]/li/div/ul/li[2]/strong').text)
                        self.driver.find_element(By.XPATH, '//*[@id="searchNum"]/option[4]').click()
                        if (record_no < 1) :
                            print(name, " 공훈록 정보 없음")
                            birth_error = False
                            self.driver.switch_to.window(self.driver.window_handles[0])
                            
                        elif (record_no == 1 and birth_error != True) :
                            self.driver.find_element(By.XPATH, '//*[@id="namekoDiv0"]').click()
                            self.heroMaker(name, fbirth)
                            
                        elif (record_no == 1 and birth_error == True) :
                            self.driver.find_element(By.XPATH, '//*[@id="namekoDiv0"]').click()
                            print('error on birth, but record found ',name)
                            self.heroMaker(name,fbirth)
                            
                        elif (record_no > 1 and birth_error == True) :
                            print(name, ' 생년월일 error, pass')
                            self.driver.switch_to.window(self.driver.window_handles[0])
                            
                        elif (record_no > 1 and birth_error != True) :
                            
                            hero_ls = {}
                            b_no = 0
                            
    
                            for i in range(record_no) :

                                path = '//*[@id="subcontents"]/div/div/div[2]/table/tbody/tr[' + str(i + 1) + ']/td[4]'
                                namepath = '//*[@id="namekoDiv'+ str(i) +'"]'
                                pagename = self.driver.find_element(By.XPATH, namepath).text
                                if (pagename == name and fbirth[:4] in (self.driver.find_element(By.XPATH, path)).text) :
                                    hero_ls[b_no] = namepath
                                    b_no += 1
                                elif (pagename == name and fbirth[-8:-4] in (self.driver.find_element(By.XPATH, path)).text) :
                                    hero_ls[b_no] = namepath
                                    b_no += 1

                                
                            if b_no < 1 :
                                print(name, " 생년월일 오류")
                                self.driver.switch_to.window(self.driver.window_handles[0])
                            elif b_no > 1 :
                                print(name, '동명동년')
                                self.driver.switch_to.window(self.driver.window_handles[0])
                            else :
                                self.driver.find_element(By.XPATH, hero_ls[0]).click()
                                self.heroMaker(name, fbirth)
                                
                        self.driver.back() 
                    
                    except :
                        print(name)
                        
                    
                self.driver.back()
                
            xpath = '//*[@id="container"]/div[3]/div[3]/div[3]/div[1]/a[' + str(_ % 10 + 3) + ']'
            
            if (_ + 1) % 10 == 0 :
                self.driver.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[3]/div[3]/div[1]/a[12]').click()
            else :
                self.driver.find_element(By.XPATH, xpath).click()

            
        
    def monumentCheck(self) :
        return 0
    
    def valid_birth(self, string):
        try : 
            int(string[:4])
            result = 'bo'
        except :
            result = 'be'
            
        try :
            int(string[-11:-7])
            result += ' do'
        except :
            result += ' de'
            
        return result
    
    def birth_definer(self, v_birth, birth):
        
        if v_birth == 'bo do':
            
            result = "".join(birth.replace(' ','').replace('~','').replace('.',''))
            return result
        
        elif v_birth == 'bo de':
            
            result = "".join(birth.replace(' ','').replace('~','').replace('.','')[:8]) + '19450101'
            return result
        
        elif v_birth == 'be do':
                
            result = '미상' + "".join(birth.replace(' ','').replace('~','').replace('.','')[-8:])
            return result
        
        elif v_birth == 'be de':
                
            result = '미상' + '19450101'
            return result
            
    def hero_definer(self, dyear, dmonth, dday) :
        
        if(dyear < 1945):
            return True
        elif(dyear == 1945 and dmonth < 8) :
            return True
        elif(dyear == 1945 and dmonth == 8 and dday <=15) :
            return True
        else :
            return False      
    
    def contents_maker(self, name) :
        
        self.driver.execute_script('window.open("about:blank", "_blank");')
        self.driver.switch_to.window(self.driver.window_handles[3])
        search_url = 'https://www.youtube.com/results?search_query='+ name +'+독립'
        self.driver.get(search_url)
        
        select = input('몇 번째 영상을 이용하시겠습니까?')
        
        if select == '5' : 
            print('youtube에 자료가 없습니다.')
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[2])
            return False
        
        else :
            
            video_path = '/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer['\
                + str(select) + ']/div[1]/ytd-thumbnail'

            self.driver.find_element(By.XPATH, video_path).click()
            
            time.sleep(3)
            video_name = self.driver.find_element(By.XPATH, '//*[@id="container"]/h1/yt-formatted-string').text
            video_url = self.driver.current_url
            
            try :
                video_desc = self.driver.find_element(By.XPATH, '//*[@id="description"]/yt-formatted-string').text + '.'
            except :
                video_desc = video_name
                
                
            self.driver.get('http://175.114.94.38/adm/object/objectList.do')
            self.driver.find_element(By.XPATH, '//*[@id="regist"]').click()
            self.driver.find_element(By.XPATH, '//*[@id="contdivcdNW204"]').click()
            
            self.driver.find_element(By.XPATH, '//*[@id="title"]').send_keys(video_name)
            self.driver.find_element(By.XPATH, '//*[@id="upThumbFile"]').send_keys('C:/Users/USER/Desktop/Duane/Python/MVPA/인물_기본.png')
            self.driver.find_element(By.XPATH, '//*[@id="linktitle1"]').send_keys(video_name)
            self.driver.find_element(By.XPATH, '//*[@id="link1"]').send_keys(video_url)
            self.driver.find_element(By.XPATH, '//*[@id="thumburl"]').send_keys(video_url)
            time.sleep(2)
            self.driver.switch_to.frame("tx_canvas_wysiwyg")
            self.driver.find_element(By.XPATH, '/html/body').send_keys(video_desc)
            self.driver.switch_to.window(self.driver.window_handles[3])
            time.sleep(2)
            self.driver.find_element(By.XPATH, '//*[@id="save"]').click()
            self.alert.accept()
            time.sleep(2)
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[2])
            return True

In [9]:
MvpaAdmin()


1 152
2 151
3 150
4 149
5 148
6 147
birth error! on  현천묵
error on birth, but record found  현천묵
7 146
8 145
9 144
10 143
11 142
12 141
13 140
14 139
15 138
16 137
17 136
18 135
19 134
20 133
21 132
22 131
23 130
24 129
25 128
26 127
27 126
28 125
29 124
30 123
31 122
32 121
33 120
34 119
35 118
36 117
37 116
38 115
39 114
40 113
41 112
42 111
43 110
44 119
45 118
46 117
47 116
48 115
49 114
50 113
51 102
52 121
53 120
54 119
55 118
56 117
57 116
58 115
59 124
60 123
61 92
62 91
63 90
64 89
65 88
66 87
67 86
68 85
69 84
70 83
71 192
72 191
73 190
74 189
75 188
76 187
77 186
78 185
79 184
80 183
81 172
82 171
83 170
84 169
85 168
86 167
87 166
88 165
89 164
90 163
91 162
92 161
93 160
94 159
95 158
96 157
97 156
98 155
99 154
100 153
101 152
102 151
103 150
104 149
105 148
106 147
birth error! on  현천묵
error on birth, but record found  현천묵
107 146
108 145


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="listForm"]/div/table/tbody/tr[9]/td[2]"}
  (Session info: chrome=98.0.4758.102)
